In [118]:
using OSQP
using SparseArrays
using ForwardDiff
using LinearAlgebra

In [364]:
P = [4. 1.; 2. 2.]
q = [1.; 10.]
A = [1. 1.; 1. 0.; 0. 1.]
l = [9.; 9.; 10.]
u = [100.5; 10.7; 10.7]

m = size(A,1)
n = size(A,2)

function osqp_solve(P,q,A,l,u)
    prob = OSQP.Model()
    OSQP.setup!(prob; P=sparse(P), q=q, A=sparse(A), l=l, u=u, alpha=1., verbose=0, eps_abs=1e-8, eps_rel=1e-8)
    results = OSQP.solve!(prob)
    
    results
end

function diff_osqp(x::AbstractArray{T}) where T
    P = reshape(x[1:n^2],n,n)
    q = x[n^2+1:n^2+n]
    A = reshape(x[n^2+n+1:n^2+n+n*m],m,n)
    l = x[n^2+n+n*m+1:n^2+n+n*m+m]
    u = x[n^2+n+n*m+m+1:n^2+n+n*m+m+m]
    
    if (T<:ForwardDiff.Dual)
        Pv = map(ForwardDiff.value, P)
        qv = map(ForwardDiff.value, q)
        Av = map(ForwardDiff.value, A)
        lv = map(ForwardDiff.value, l)
        uv = map(ForwardDiff.value, u)
        
        results = osqp_solve(Pv,qv,Av,lv,uv)
        xv = results.x
        λ_plus = max.(0., results.y)
        λ_minus = min.(0., results.y)
        
        Gv = vcat(Av, -Av)
        hv = vcat(uv, -lv)
        λv = vcat(λ_plus, -λ_minus)
        
        M = vcat(hcat(Pv, Gv'),
                 hcat(Diagonal(λv)*Gv, Diagonal(Gv*xv - hv)))
                
        Pp = map(ForwardDiff.partials, P)
        qp = map(ForwardDiff.partials, q)
        Ap = map(ForwardDiff.partials, A)
        Apt = Ap[reshape(1:m*n,(m,n))']
        lp = map(ForwardDiff.partials, l)
        up = map(ForwardDiff.partials, u)
        
        Gp = vcat(Ap, -Ap)
        Gpt = hcat(Apt, -Apt)
        hp = vcat(up, -lp)
                
        dres = vcat(Pp*xv + qp + Gpt*λv,
                    Diagonal(λv)*(Gp*xv - hp)) 
        
        xp = (-pinv(Matrix(M)) * dres)[1:n]
                
        sol = map(T, xv, xp)
    else    
        results = osqp_solve(P,q,A,l,u)
        sol = results.x
    end
    
    sol
end

x0 = vcat(P[:],q[:],A[:],l[:],u[:]);

In [355]:
# numerical
sol = diff_osqp(x0)
ϵ = sqrt(eps(1.))
J_num = zeros(n,length(x0))
for i = 1:length(x0)
    δ = zeros(length(x0))
    δ[i] = ϵ
    J_num[:,i] = (diff_osqp(x0 + δ) .- sol)/ϵ
end

J_num

2×18 Array{Float64,2}:
 -3.57628e-7  0.0  -7.15256e-7  …   1.0         0.0  0.0  0.0  0.0
 -1.19209e-7  0.0   0.0            -1.19209e-7  1.0  0.0  0.0  0.0

In [365]:
cfg1 = ForwardDiff.JacobianConfig(diff_osqp, x0, ForwardDiff.Chunk{length(x0)}())
J_auto = ForwardDiff.jacobian(diff_osqp, x0, cfg1)

6-element Array{ForwardDiff.Partials{18,Float64},1}:
 Partials(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)                  
 Partials(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)                  
 Partials(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)                  
 Partials(-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0)
 Partials(-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0)
 Partials(-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0)

6-element Array{ForwardDiff.Partials{18,Float64},1}:
 Partials(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)                  
 Partials(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)                  
 Partials(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)                  
 Partials(-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0)
 Partials(-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0)
 Partials(-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0)

2×18 Array{Float64,2}:
  2.11511e-8   3.43706e-17   2.35013e-8   …  2.35013e-9  0.0  0.0  0.0
 -1.99084e-15  1.02409e-8   -2.21205e-15     1.0         0.0  0.0  0.0

In [357]:
# dq
display(J_num[:,1])
display(J_auto[:,1])

2-element Array{Float64,1}:
 -3.5762786865234375e-7
 -1.1920928955078125e-7

2-element Array{Float64,1}:
  2.1151132700292964e-8
 -1.990842250319133e-15

In [358]:
maximum(abs.(J_num - J_auto))

1.718557875562965e-6

In [359]:
sol

2-element Array{Float64,1}:
 8.999999889544085
 9.999999955622787

In [ ]:
#         results = osqp_solve(Pv,qv,Av,lv,uv)
#         solv = results.x
#         y = results.y
#         y_plus = max.(0., y)
#         y_minus = min.(0., y)
#         z = Av * solv
#         M = vcat(hcat(Pv, Av', zeros(n, n)),
#                  hcat(Av, zeros(m, n), -Matrix{Float64}(I, m, m)),
#                  hcat(zeros(1, 1), ((z - uv).*(y .>= 0.))', y_plus'),
#                  hcat(zeros(1, 1), ((z - lv).*(y .<= 0.))', y_minus'))
        
#         dres = vcat(Pp*solv + qp + Apt*y,
#                     Ap*solv,
#                     -sum(y_plus .* up, dims=1),
#                     -sum(y_minus .* lp, dims=1))